In [3]:
import os
import re

In [4]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import pandas as pd

In [5]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [6]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [7]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [8]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [9]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [10]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get KFC Data

In [16]:
scraper = cloudscraper.create_scraper()

In [17]:
def get_coords(url: str) -> Point:
    r = scraper.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    latitude_meta = soup.find("meta", {"itemprop": "latitude"})
    longitude_meta = soup.find("meta", {"itemprop": "longitude"})
    lat = float(latitude_meta.attrs["content"])
    lon = float(longitude_meta.attrs["content"])
    return Point(lon, lat)

In [19]:
store_list = []
url_base = "https://locations.kfc.com/"
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    # Get State code
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"].lower()

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    # Get all locations in a state
    url = os.path.join(url_base, state_code)
    r = scraper.get(url)

    if r.status_code != 200:
        print(states_df.iloc[i]["NAME"])
        continue

    soup = BeautifulSoup(r.text, "html.parser")
    locations_as = soup.find_all("a", {"class": "Directory-listLink"})

    # Parse all locations in a state
    for locations_a in tqdm(locations_as, desc=f"Parsing Locations In {state_code}"):
        href = locations_a.attrs["href"]
        count = int(re.findall(r"\d+", locations_a.attrs["data-count"])[0])
        location_url = url_base + href

        if count == 1:
            point = get_coords(location_url)
            store_dict = {
                "STATE": state_code,
                "geometry": point,
            }
            store_list.append(store_dict)
            state_store_list.append(store_dict)
        else:
            r_location = scraper.get(location_url)
            location_soup = BeautifulSoup(r_location.text, "html.parser")
            locations_as = location_soup.find_all("a", {"class": "Teaser-titleLink"})

            # Multiple stores in locations, parse the stores per location
            for location_a in locations_as:
                try:
                    href = location_a.attrs["href"]
                    location_url = os.path.join(url_base, href.replace("../", ""))
                    point = get_coords(location_url)
                    store_dict = {
                        "STATE": state_code,
                        "geometry": point,
                    }
                    store_list.append(store_dict)
                    state_store_list.append(store_dict)
                except Exception as e:
                    print(location_url)

    if state_store_list:
        kfc_state_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        kfc_state_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Puerto Rico
American Samoa
United States Virgin Islands
Guam
Commonwealth of the Northern Mariana Islands


Parsing Locations In in:   0%|          | 0/78 [00:00<?, ?it/s]

https://locations.kfc.com/in/indianapolis/8456-michigan-rd-H740024_D


Parsing Locations In mn:   0%|          | 0/40 [00:00<?, ?it/s]

Parsing Locations In ct:   0%|          | 0/33 [00:00<?, ?it/s]

#### If the above cells breaks just keep running until complete. Then run the line below

In [20]:
kfc_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        state_kfc_gdf = gpd.read_file(constructed_path)
        kfc_state_gdfs.append(state_kfc_gdf)

In [21]:
kfc_state_gdf = gpd.GeoDataFrame(pd.concat(kfc_state_gdfs, ignore_index=True))

In [22]:
kfc_state_gdf = kfc_state_gdf.to_crs(9311)
kfc_state_gdf.to_file(f"data/stores.gpkg")

## Combine With States

In [32]:
kfc_state_counts_series = kfc_state_gdf.groupby("STATE").size()
kfc_state_counts_df = pd.DataFrame(
    {
        "STUSPS": kfc_state_counts_series.index,
        "stores": kfc_state_counts_series.values,
    }
)

In [34]:
kfc_state_counts_df["STUSPS"] = kfc_state_counts_df["STUSPS"].str.upper()

In [36]:
kfc_state_counts_df = states_with_population_df.merge(
    kfc_state_counts_df, on="STUSPS", how="left"
)

In [37]:
kfc_state_counts_df = kfc_state_counts_df.fillna(0)
kfc_state_counts_df["stores"] = kfc_state_counts_df["stores"].astype(int)

In [38]:
kfc_state_counts_df["per_1000"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 1000
)
kfc_state_counts_df["per_10k"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 10_000
)
kfc_state_counts_df["per_100k"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 100000
)
kfc_state_counts_df["per_500k"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 500_000
)
kfc_state_counts_df["per_1m"] = kfc_state_counts_df["stores"] / (
    kfc_state_counts_df["POPULATION"] / 1_000_000
)

In [39]:
kfc_state_counts_df = kfc_state_counts_df.dropna()

In [40]:
kfc_state_counts_df = kfc_state_counts_df.to_crs(9311)
kfc_state_counts_df.to_file(f"data/stores_by_states.gpkg")